# Load Packages

What we can recall from critisicm regarding this presentation:

- did not take the equal weighted as benchmark
- use total return indices (I think)

In [7]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

# Seaborn
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

# little function for later
def isNaN(string):
    return string != string

# Load Data

In [8]:
# rawdata is direct import from excel

# Import of excel data, sheet by sheet
xls_ETF = pd.ExcelFile("1_Data/Data Clean TM/Data_ETFxIndex_stiched.xlsx")
xls_Inflation = pd.ExcelFile("1_Data/Data Clean TM/Data_Inflation.xlsx")
xls_Yield = pd.ExcelFile("1_Data/Data Clean TM/Yields_Clean_TM.xlsx")
xls_FX = pd.ExcelFile("1_Data/Data Clean TM/Data_FX_spot.xlsx")

ETF_raw = pd.DataFrame(pd.read_excel(xls_ETF))
Inflation_raw = pd.DataFrame(pd.read_excel(xls_Inflation, 2))
Yield_long_raw = pd.DataFrame(pd.read_excel(xls_Yield, 2))
Yield_short_raw = pd.DataFrame(pd.read_excel(xls_Yield, 4))
FX_raw = pd.DataFrame(pd.read_excel(xls_FX, 3))

# keep a safe copy of the rawdata to compare the changes
ETF = ETF_raw.copy()
Inflation = Inflation_raw.copy()
Yield_long = Yield_long_raw.copy()
Yield_short = Yield_short_raw.copy()
FX = FX_raw.copy()

# Set date as index
ETF.set_index("Datum", inplace=True)
Inflation.set_index("Datum", inplace=True)
Yield_long.set_index("Datum", inplace=True)
Yield_short.set_index("Datum", inplace=True)
FX.set_index("Dates", inplace=True)

In [9]:
# Drop all the columns in which only text is and safe them separately --> ONLY RUN ONCE
# ETF_text = ETF.iloc[0,:]
# Yield_text = Yield.iloc[0,:]
# Yield_short_text = Yield_short.iloc[0,:]
# cpicore_text = cpicore.iloc[0:1,:]
# ETF.drop([0], inplace = True)
# Yield.drop([0], inplace = True)
# Yield_short.drop([0], inplace = True)
# cpicore.drop([0,1], inplace = True)

# Value

Recent infl – the average of the headline and core annual CPI inflation rate
--> we don't have headline inflation


Effective target infl - The effective inflation target is the mean of the target range announced or implied by the authorities plus an adjusted for past “target misses”, which is the last 3 years’ average gap between actual inflation and the target means
--> we don't have target rates for all countries --> use 2.5 as target rate

Formula: (1/n)*recent infl + ((n-1)/n)*effective target infl

In [10]:
# first we will take the difference between the effective target (mean) inflation and the actual inflation

cpiheadline = Inflation.copy()

eff_target_diff = pd.DataFrame(columns = cpiheadline.columns, index = cpiheadline.index)

for i in eff_target_diff.columns:
    if i == "Australia":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.025
    elif i == "Canada":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "China":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Germany":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "France":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "UK":
        eff_target_diff.loc[:"2003-11-28",i] = cpiheadline.loc[:"2003-11-28",i] - 0.025
        eff_target_diff.loc["2003-12-31":,i] = cpiheadline.loc["2003-12-31":,i] - 0.02
    elif i == "Italy":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Japan":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.033
    elif i == "Singapore":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "USA":
        eff_target_diff.loc[:"2011-12-31",i] = cpiheadline.loc[:"2011-12-31",i] - 0.0185
        eff_target_diff.loc["2012-01-31":,i] = cpiheadline.loc["2012-01-31":,i] - 0.02
    elif i == "Spain":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Switzerland":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.01
    elif i == "India":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.04

In [11]:
np.nanmean(eff_target_diff.loc[:"2002-11-30", "Singapore"])
#np.nanmean(eff_target_diff.loc["2020-05-31":, "Singapore"])

-0.012722222222222225

In [12]:
# create the rolling 3-year average of the core inflation

# first I have to set up an empty dictionary to store the rolling averages
cpiheadline_avg = {}
for i in cpiheadline.columns:
    cpiheadline_avg[i] = []

# next up we iterate over the cpicore data to get the index and safe them in the dictionary
runner = len(cpiheadline.index)
for i in cpiheadline.index:
    if runner == 35:
        break
    
    for j,k in enumerate(cpiheadline.columns):
        inflation = np.nanmean(eff_target_diff.iloc[runner-36:runner, j])
        cpiheadline_avg[k].insert(0, inflation)
    runner -= 1
    
#cpiheadline_avg
cpiheadline_avg_trimmed = pd.DataFrame(cpiheadline_avg)

AttributeError: 'float' object has no attribute 'dtype'

In [ ]:
cpiheadline_avg_trimmed # until here I checked several times whether it is correct and it seems to work

In [ ]:
# now we calculate the value formula:
# (1/n)*recent inflation + ((n-1)/n)*effective target inflation for the years: 2, 5, 10, 20, 30

# first we need to align the dataframes 
cpiheadline_trimmed = cpiheadline[35:]
cpiheadline_avg_trimmed.set_index(cpiheadline_trimmed.index, inplace = True)

# second we create an empty dictionary to fill with the calculations
formula_values = {}
x = 0
for i in cpiheadline_trimmed.columns:
    x += 1
    for j in [2,5,10,20,30]:
        formula_values[f"{i},{j}"] = []
formula_values = pd.DataFrame(columns = formula_values.keys(), index = cpiheadline_trimmed.index)


# now that we have the empty dictionary to fill, we start with caluclating the inflation expectation
for i,k in enumerate (cpiheadline_trimmed.columns):
    for j,m in enumerate([2,5,10,20,30]):
        formula_values.iloc[:,j+i*5] = (1/m)*cpiheadline_trimmed[k] +((m-1)/m)*cpiheadline_avg_trimmed[k]
        
#formula_values

<ipython-input-5-90c02453a451>:21: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  formula_values.iloc[:,j+i*5] = (1/m)*cpiheadline_trimmed[k] +((m-1)/m)*cpiheadline_avg_trimmed[k]


In [ ]:
# now we have the issue that the Yield_clean data and the Inflation_clean data doesn't have the same sequence of countries
# to avoid this we resort the Yield_clean data to fit the Inflation_clean data

clean_names = ["Germany", "France","Spain","Italy","United Kingdom", "Switzerland", "China", "Australia", "Japan", "USA", "Canada", "Singapore", "India"]

# we split the dataframe into the different countries...
Germany = Yield_long.iloc[:,:5]
France = Yield_long.iloc[:,5:10]
Spain = Yield_long.iloc[:,10:15]
Italy = Yield_long.iloc[:,15:20]
UK = Yield_long.iloc[:,20:25]
Switzerland = Yield_long.iloc[:,25:30]
China = Yield_long.iloc[:,30:35]
Australia = Yield_long.iloc[:,35:40]
Japan = Yield_long.iloc[:,40:45]
USA = Yield_long.iloc[:,45:50]
Canada = Yield_long.iloc[:,50:55]
Singapore = Yield_long.iloc[:,55:60]
India = Yield_long.iloc[:,60:]

# ... and fusion them together to have the same order as the Inflation data (plus dividing by 100 to have same level as Inflation)
Yields_aligned = [India, Singapore, Canada, USA, Japan, Australia, China, Switzerland, UK, Spain, France, Germany, Italy]
Yields_aligned = pd.concat(Yields_aligned, axis=1)
Yields_aligned = Yields_aligned /100
#Yields_aligned

In [ ]:
# in a next step we adjust the yield data to have the same lookback as the other data

Yield_trimmed = Yields_aligned[35:]

Real_yield = pd.DataFrame(0, columns = formula_values.columns, index = formula_values.index)

# now we take the difference between yield etf and inflation
for i,k in enumerate(formula_values.columns):
    for j,l in enumerate(formula_values.index):
        Real_yield.iloc[j,i] = Yield_trimmed.iloc[j,i] - formula_values.iloc[j,i]
#Real_yield

,"India,2","India,5","India,10","India,20","India,30","Singapore,2","Singapore,5","Singapore,10","Singapore,20","Singapore,30",...,"Germany,2","Germany,5","Germany,10","Germany,20","Germany,30","Italy,2","Italy,5","Italy,10","Italy,20","Italy,30"
Datum,,,,,,,,,,,,,,,,,,,,,
2002-11-30,NaN,NaN,NaN,NaN,NaN,0.018411,0.033228,0.043260,0.048026,0.047785,...,0.027169,0.039412,0.048420,0.055769,0.055292,0.016012,0.030556,0.041020,0.048712,0.050070
2002-12-31,NaN,NaN,NaN,NaN,NaN,0.015450,0.030030,0.038690,0.043000,0.042950,...,0.023987,0.036127,0.046050,0.054032,0.053352,0.011091,0.026158,0.037840,0.046091,0.046691
2003-01-31,NaN,NaN,NaN,NaN,NaN,0.013929,0.028562,0.037910,0.043444,0.043372,...,0.022910,0.035090,0.045060,0.052970,0.052723,0.010200,0.024520,0.035930,0.044320,0.045487
2003-02-28,NaN,NaN,NaN,NaN,NaN,0.010492,0.025676,0.035370,0.041102,0.041166,...,0.019736,0.032159,0.043360,0.052876,0.052797,0.007761,0.021839,0.034185,0.044548,0.045976
2003-03-31,NaN,NaN,NaN,NaN,NaN,0.013517,0.025316,0.034285,0.039780,0.039785,...,0.020073,0.033593,0.044330,0.053988,0.054024,0.009063,0.023684,0.035655,0.045385,0.047035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.025409,0.041948,0.047797,0.050627,0.051907,-0.004613,0.015897,0.023770,0.024462,0.023219,...,-0.039996,-0.012138,-0.001805,0.004586,0.004507,-0.034808,0.004357,0.022335,0.030249,0.029747
2022-12-31,0.029811,0.043652,0.048602,0.050983,0.052113,-0.005461,0.014541,0.022245,0.022852,0.021581,...,-0.039232,-0.013316,-0.003630,0.002438,0.002250,-0.036392,0.001823,0.019485,0.027241,0.026686
2023-01-31,0.030838,0.044334,0.049170,0.051493,0.052604,-0.005252,0.013674,0.021020,0.021448,0.020117,...,-0.033218,-0.012093,-0.004005,0.001264,0.000811,-0.036933,-0.000243,0.016910,0.024412,0.023772


In [ ]:
# in a final step we will average over the duration of the countries to get one average for each country

Real_yield_avg = pd.DataFrame(columns = cpiheadline_avg_trimmed.columns, index = Real_yield.index)

runner = 0
for i in range(0, len(Yields_aligned.columns)-1, 5):
    temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))
    Real_yield_avg.iloc[:,runner] = temp.values
    runner +=1
    
#Real_yield_avg

<ipython-input-19-32f040c4ca01>:7: RuntimeWarning: Mean of empty slice
  temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))
<ipython-input-19-32f040c4ca01>:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Real_yield_avg.iloc[:,runner] = temp.values
<ipython-input-19-32f040c4ca01>:7: RuntimeWarning: Mean of empty slice
  temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))


,India,Singapore,Canada,USA,Japan,Australia,China,Switzerland,UK,Spain,France,Germany,Italy
Datum,,,,,,,,,,,,,
2002-11-30,NaN,0.038142,0.040897,0.049712,0.045416,0.040049,NaN,0.023939,0.054983,0.026126,0.043910,0.045212,0.037274
2002-12-31,NaN,0.034024,0.035837,0.044641,0.043797,0.035000,NaN,0.021170,0.051624,0.022888,0.040268,0.042709,0.033574
2003-01-31,NaN,0.033443,0.038607,0.045030,0.042133,0.035061,NaN,0.021291,0.051239,0.021023,0.038475,0.041751,0.032091
2003-02-28,NaN,0.030761,0.036602,0.042970,0.042106,0.033406,NaN,0.020968,0.049642,0.020183,0.037702,0.040185,0.030862
2003-03-31,NaN,0.030536,0.037642,0.039602,0.040662,0.035846,NaN,0.023387,0.050895,0.021027,0.037697,0.041202,0.032165
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.043538,0.016547,0.010547,0.021676,0.021544,0.022938,0.023062,0.006303,0.002752,0.005886,0.013793,-0.008969,0.010376
2022-12-31,0.045032,0.015152,0.009672,0.021593,0.020613,0.021749,0.024608,0.005594,0.001354,0.005306,0.012604,-0.010298,0.007769
2023-01-31,0.045688,0.014201,0.009618,0.021692,0.019527,0.019785,0.024872,0.005352,-0.000396,0.006128,0.012105,-0.009448,0.005583


# Momentum

In [ ]:
# Translate into python


In [15]:
# first we take the average of the three Swiss indices

ETF_price = ETF.copy()

# s1 = ETF_price["Switzerland 1"]
# s2 = ETF_price["Switzerland 2"]
# s3 = ETF_price["Switzerland 3"]

#ETF_price.drop(["Switzerland 2", "Switzerland 3"], axis = 1, inplace = True)
#ETF_price.loc["2003-11-28":,"Switzerland 1"] = (np.nanmean([s1.loc["2003-11-28":], s2.loc["2003-11-28":], s3.loc["2003-11-28":]], axis = 0))
#ETF_price.rename(columns={"Switzerland 1": "Switzerland"}, inplace = True)
#ETF_price

In [23]:
# Create Monthly Returns out of ETF Prices
ETF_returns = ETF_price.pct_change()
ETF_ret = ETF_returns.tail(-1)
ETF_ret

In [32]:
# Create Weights for Momentum

# Prep Work
nAssets = len(ETF_price.columns) # 13 Countries
nMonths = len(ETF_price) # 279 Months of returns (31.01.2000 - 31.03.2023)
lookbackStart = 12
lookbackEnd = 1
firstMonth = lookbackStart + 1 # we can only start computing weights in 13.month bc we have 12 months lookback 
nLongs = 5
nShorts = 5
momLongWeights = np.zeros((nMonths, nAssets))
momLSWeights = np.zeros((nMonths, nAssets))



In [31]:
momLongWeights.shape

(279, 13)

In [ ]:
for month in range(firstMonth, nMonths)
    pastReturns = np.divide(ETF_price[month - lookbackEnd, :], ETF_price[month - lookbackStart, :])-1
    momLSWeights(month, :) = computeSortWeights(pastReturns, nLongs, nShorts, 1);


In [ ]:
# Data Cleanup -> Returns in sync with weights!

In [ ]:
# we start off by calculating the returns
ETF_returns = ETF_price.pct_change()

# here we convert the values to numeric in order to take the maximum afterwards
ETF_returns = ETF_returns.apply(pd.to_numeric)

# here we aggregate the past 11 months returns
past_11 = (ETF_returns+1).rolling(11).apply(np.prod)-1 # accumulate returns over 11 months 
past_11

# Carry

In [ ]:
Yield_short

,Germany,France,Spain,Italy,UK,Switzerland,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
1999-12-31,3.140,3.07,3.500,3.250,5.630,1.750,NaN,4.910,0.010,6.320,4.950,0.750,9.970
2000-01-31,3.440,3.31,3.410,3.380,5.790,2.080,NaN,5.400,0.110,6.570,5.330,1.560,9.910
2000-02-29,3.600,3.62,3.620,3.580,5.860,2.440,NaN,4.960,0.010,6.570,5.380,1.690,9.840
2000-03-31,3.740,3.77,3.640,3.830,6.050,2.940,NaN,5.610,0.010,6.800,5.320,2.240,9.970
2000-04-28,3.970,3.96,3.850,4.030,5.860,3.180,NaN,6.110,0.010,6.880,5.640,2.310,9.160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,1.532,1.69,1.715,2.101,2.831,0.291,1.602,2.662,-0.199,4.846,4.058,3.491,6.681
2022-12-30,1.532,1.69,1.715,2.101,2.831,0.291,1.602,2.662,-0.199,4.846,4.058,3.491,6.681
2023-01-31,1.532,1.69,1.715,2.101,2.831,0.291,1.602,2.662,-0.199,4.846,4.058,3.491,6.681


In [ ]:
# for carry we will now calculate the differences between the long-term yields (2,5,10,20,30) and short-term yields (1M, 3M)
# we will create a dataframe for the 1M and the 3M

Carry_3m = pd.DataFrame(columns = Yield_long.columns, index = Yield_long.index)
avg_Carry_3m = pd.DataFrame(0, columns = Yield_short.columns, index = Yield_short.index)

# next we will iterate through all the columns of the yields and substract the short-term rate
runner = 0
counter = 0
for i in range(0,len(Yield_short.columns)-1):
    if counter == 5:
        counter = 0
        runner += 2
    counter += 1
    Carry_3m.iloc[:,i] = Yield_long.iloc[:,i] - Yield_short.iloc[:,runner+1]
    

for i in range(0,len(Yield_short.columns)-1):
    for j in range(5):
        Carry_3m.iloc[:,i+j] = Yield_long.iloc[:,i+j] - Yield_short.iloc[:,i]
    

# # next we will equal weight the carry signal among the countries
# Carry_1m.iloc[2:66,30:35] = 0

# runner = 0
# for i in range(0,len(Carry_1m.columns)-1, 5):
#     avg_Carry_1m.iloc[:,runner] = np.nanmean([Carry_1m.iloc[:,i], Carry_1m.iloc[:,i+1], Carry_1m.iloc[:,i+2], Carry_1m.iloc[:,i+3], Carry_1m.iloc[:,i+4]], axis = 0)
#     avg_Carry_3m.iloc[:,runner] = np.nanmean([Carry_3m.iloc[:,i], Carry_3m.iloc[:,i+1], Carry_3m.iloc[:,i+2], Carry_3m.iloc[:,i+3], Carry_3m.iloc[:,i+4]], axis = 0)
#     runner += 1
    
    
    
# # change data values from object to float
# avg_Carry_1m = avg_Carry_1m.apply(pd.to_numeric)
# avg_Carry_3m = avg_Carry_3m.apply(pd.to_numeric)


0
1
2
3
4
1
2
3
4
5
2
3
4
5
6
3
4
5
6
7
4
5
6
7
8
5
6
7
8
9
6
7
8
9
10
7
8
9
10
11
8
9
10
11
12
9
10
11
12
13
10
11
12
13
14
11
12
13
14
15


<ipython-input-46-25dc951a7823>:21: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Carry_3m.iloc[:,i+j] = Yield_long.iloc[:,i+j] - Yield_short.iloc[:,i]


In [ ]:
Carry_3m

,Germany,Germany.1,Germany.2,Germany.3,Germany.4,France,France.1,France.2,France.3,France.4,...,Singapore,Singapore.1,Singapore.2,Singapore.3,Singapore.4,India,India.1,India.2,India.3,India.4
Datum,,,,,,,,,,,,,,,,,,,,,
1999-12-31,1.124,1.922,1.929,2.922,0.484,2.532,NaN,0.724,6.351,-0.168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-31,1.033,1.941,2.204,2.853,0.357,2.433,NaN,0.399,6.268,-0.370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,0.964,1.549,1.963,2.424,0.068,2.089,NaN,0.776,6.251,-0.490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,0.759,1.187,1.644,1.913,-0.391,1.557,NaN,-0.180,5.931,-1.028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-28,0.687,1.146,1.519,1.804,-0.241,1.454,NaN,-0.624,5.893,-1.106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.367,0.294,0.402,0.205,-0.683,1.793,0.685,-0.023,3.290,-1.819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-30,0.367,0.294,0.402,0.205,-0.683,1.793,0.685,-0.023,3.290,-1.819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31,0.367,0.294,0.402,0.205,-0.683,1.793,0.685,-0.023,3.290,-1.819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# for carry we will now calculate the differences between the long-term yields (2,5,10,20,30) and short-term yields (1M, 3M)
# we will create a dataframe for the 1M and the 3M

Carry_1m = pd.DataFrame(columns = Yield.columns, index = Yield.index)
Carry_3m = pd.DataFrame(columns = Yield.columns, index = Yield.index)
avg_Carry_1m = pd.DataFrame(0, columns = countries, index = Yield.index)
avg_Carry_3m = pd.DataFrame(0, columns = countries, index = Yield.index)

# next we will iterate through all the columns of the yields and substract the short-term rate
runner = 0
counter = 0
for i,k in enumerate(Yield.columns):
    if counter == 5:
        counter = 0
        runner += 2
    counter += 1
    Carry_1m.iloc[:,i] = Yield.iloc[:,i] - Yield_short.iloc[:,runner]
    Carry_3m.iloc[:,i] = Yield.iloc[:,i] - Yield_short.iloc[:,runner+1]
    
# next we will equal weight the carry signal among the countries
for i in range(len(countries)):
    j = 0
    while j != 5:
        avg_Carry_1m.iloc[:,i] += 1/5 * Carry_1m.iloc[:,i+j]
        avg_Carry_3m.iloc[:,i] += 1/5 * Carry_3m.iloc[:,i+j]
        j += 1
        
# change data values from object to float
avg_Carry_1m = avg_Carry_1m.apply(pd.to_numeric)
avg_Carry_3m = avg_Carry_3m.apply(pd.to_numeric)

In [ ]:
Carry_1m.iloc[2:66,30:35] = 1
Yield.iloc[2:80,30:35]

In [ ]:
x = np.nanmean([Carry_1m.iloc[0:2,0], Carry_1m.iloc[0:2,1]], axis = 0)
x


In [ ]:
Carry_1m.iloc[0:2,0].sub(np.nan, fill_value = 0)

# Sort function to create the buckets

In [ ]:
def pmp_sort(pandas_DataFrame, nMax):
    top_bucket = {}
    bottom_bucket = {}
    for i,k in enumerate(pandas_DataFrame.index):
        top_bucket[k] = []
        bottom_bucket[k] = []
        a = pandas_DataFrame.iloc[i]
        for j in range(5):
            x = a.idxmax()
            y = a.idxmin()
            if isNaN(x) and isNaN(y):
                break
            top_bucket[k].append(x)
            bottom_bucket[k].append(y)
            a = a.drop([x,y])

In [ ]:
top_bucket = {}
bottom_bucket = {}
for i,k in enumerate(avg_yield_countries.index):
    top_bucket[k] = []
    bottom_bucket[k] = []
    a = avg_yield_countries.iloc[i]
    for i in range(3):
        x = a.idxmax()
        y = a.idxmin()
        if isNaN(x):
            continue
        else:
            top_bucket[k].append(x)
            a = a.drop(x)
        if isNaN(y):
            continue
        else:
            bottom_bucket[k].append(y)
            a = a.drop(y)

In [ ]:
print(top_bucket)
print(bottom_bucket)

# avg_yield_countries

In [ ]:
x = avg_Carry_1m.iloc[0].drop(["Switzerland", np.nan])
avg_Carry_1m.iloc[0]
x

In [ ]:
a = avg_yield_countries.iloc[0,2]
b = avg_yield_countries.iloc[0,4]
np.isnan(["a",b]).all()

In [ ]:
p = []
p.append(np.nan)
p

# Cut Momentum Frame

In [ ]:
# Exclude Trading Days column and MSCI_EU Benchmark to have only Momentum columns
prices = df.loc[:,"Healthcare":"Real_estate"]
prices.dtypes

In [ ]:
# Calculating Monthly returns
mtl_ret = prices.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
mtl_ret

past_11 = (mtl_ret+1).rolling(11).apply(np.prod)-1 # accumulate returns over 11 months 
past_11.head(11)

In [ ]:
# Defining formation date <-  where portfolio gets created
formation = dt.datetime(1999, 12, 31, 0, 0)

In [ ]:
# Defining MeasurementDate <- up to last date of month before formation date
end_measurement = formation - MonthEnd(1)
end_measurement

In [ ]:
# Past 12 Month performance without most recent month <- in this case Jan, Feb, Mar, Apr, Jun, Jul, Sep, Oct, Nov but not December!
ret_12 = past_11.loc[end_measurement]
ret_12 = ret_12.reset_index()
ret_12.rename(columns={ ret_12.columns[1]: "returns" }, inplace = True)
ret_12

In [ ]:
ret_12 = ret_12.sort_values(["returns"], ascending=False)
ret_12["bucket"] = [2,2,2,2,2,1,0,0,0,0,0] # Make three buckets
ret_12

In [ ]:
# Create winners and loser list based on bucket
winners = ret_12[ret_12.bucket == 2]
losers = ret_12[ret_12.bucket == 0]
winners

In [ ]:
# Calculate the winner and loser returns over the last 12 months excluding the most recent month
winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners["index"])]
loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers["index"])]

In [ ]:
Momentumprofit = winnerret.mean() - loserret.mean()
Momentumprofit

# Functionalising this Methodology

In [ ]:
def data_cleaner(pandas_df, index="Date", prices_start='Healthcare', prices_end='Real_estate', resample_freq='M', lookback=11):
    print(pandas_df.isnull().values.any())
    df = pandas_df.set_index(index, inplace=False)
    prices = df.loc[:,prices_start:prices_end]
    mtl_ret = prices.pct_change().resample(resample_freq).agg(lambda x: (1+x).prod()-1)
    past_11 = (mtl_ret+1).rolling(lookback).apply(np.prod)-1
    return prices, mtl_ret, past_11


In [ ]:
dfff = dff.set_index("Date", inplace = False)
x = dfff.resample("M")
type(x)

In [ ]:
#test out the data_cleaner function

prices, mtl_ret, past_11 = data_cleaner(dff, lookback = 3, resample_freq= 'BM')
print(f'\nprices:\n {prices}, \nmonthly_return:\n {mtl_ret}, \npast_11:\n {past_11}')

In [ ]:
past_11.head(5)

In [ ]:
def momentum(formation, past_11, num_l_s=5):
    end_measurement = formation - MonthEnd(1)
    ret_12 = past_11.loc[end_measurement]
    ret_12 = ret_12.reset_index()
    ret_12.rename(columns={ ret_12.columns[1]: "returns" }, inplace = True)
    ret_12 = ret_12.sort_values(["returns"], ascending=False)
    
    middle = 11-2*num_l_s
    l_s = []
    for i in range(num_l_s):
        l_s.append(2)
    for i in range(middle):
        l_s.append(1)
    for i in range(num_l_s):
        l_s.append(0)
    ret_12["bucket"] = l_s # Make arbitrary number of buckets
    
    winners = ret_12[ret_12.bucket == 2]
    losers = ret_12[ret_12.bucket == 0]
    winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners["index"])]
    loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers["index"])]
    Momentumprofit = winnerret.mean() - loserret.mean()
    return Momentumprofit







In [ ]:
formation = dt.datetime(1999, 12, 31, 0, 0)
momentum(formation, past_11) # has to be identical with [35]

In [ ]:
def benchmarker(pandas_df, profits_list):
    dff = pandas_df.set_index("Date", inplace=False)
    
    benchmark = dff[["MSCI_EU"]]
    benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
    
    bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
    comparison = pd.DataFrame(bm_mtl_ret)
    comparison["mom_ret"] = profits_list
    comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
    comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
    comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
    print(comparison)
    
    fig, ax = plt.subplots()
    ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
    ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
    ax.legend(loc="best")
    ax.set_ylabel('Monthly Returns')
    ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")


In [ ]:
# Creating loop for every month  starting at formation date -> creates df with all these dates
for i in range(273): # Eig wären es 273 Monate zwischen 31.12.1999 und 31.12.2022 -> stimmt irgendwie ned ganz aber lauft jetzt halt so bis endi august
    print(formation + MonthEnd(i))

#wären es nicht 276 Monate? has mit emene Zeitspannen-rechner usgrechnet --> gahd ets bis endi november 2022
for i in range(276):
    print(formation + MonthEnd(i))


In [ ]:
# Creating loop for storing the profits and the realized dates of the momentum strategy
profits = []
dates = []

'''
for i in range(273):
    profits.append(momentum(formation + MonthEnd(i)))
    dates.append(formation + MonthEnd(i))
'''

#version 2.0 (da gahds nume bis 274 --> gid en error wenn en monet meh nimmsch)
for i in range(274):
    profits.append(momentum(formation + MonthEnd(i), past_11))
    dates.append(formation + MonthEnd(i))


In [ ]:
benchmarker(df, profits)

In [ ]:
mom = pd.DataFrame({"Dates": dates,"Profits": profits})
mom

# Benchmarking

In [ ]:
benchmark = df[["MSCI_EU"]]
benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
benchmark

In [ ]:
bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
bm_mtl_ret


In [ ]:
#compare momentum result vs benchmark
comparison = pd.DataFrame(bm_mtl_ret)
comparison["mom_ret"] = profits
comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
comparison


In [ ]:
comparison

# Comparison Plot

In [ ]:
# Plotting
fig, ax = plt.subplots()
ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
ax.legend(loc="best")
ax.set_ylabel('Monthly Returns')
ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")